In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [2]:
data = datas[1499]
question = data[question_string]
topics = data['topic_entity']
facts, paths = [], []
print(question)

when did shaq come into the nba?


In [3]:
for topic in topics:
    topic_name = topics[topic]
    relations = get_relations(question, topic, topic_name, args, 5)
    relations = relations[:1]
    paths.append({"entity:": topic_name, "relations": relations})
    entities_id, entities_name = get_entities(topic, relations)
    topic_prop = direct_propagate(question, topic_name, relations, entities_name, args)
    facts += topic_prop
prompt = question_prompt.format("\n".join(facts), question)
response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm)

output = {"question": question, "result": response, "path": paths, "prompt": facts}

In [4]:
output

{'question': 'when did shaq come into the nba?',
 'result': '  Sure, here are the possible answers to the question "when did Shaq come into the NBA?" based on the given background information:\n\n* 1992',
 'path': [{'entity:': "Shaquille O'Neal",
   'relations': ['basketball.basketball_player.player_statistics']}],
 'prompt': ["Shaquille O'Neal played in the NBA for 19 seasons, from 1992 to 2011, for several teams including the Orlando Magic, Los Angeles Lakers, Miami Heat, and Phoenix Suns."]}

In [ ]:
paths

In [ ]:
save_2_jsonl("lmp_{}_{}.jsonl".format(args.dataset, args.llm), output)